In [2]:
# !pip install --quiet  datasets #to access squad dataset
# !pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     #for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  #tokenizers from HuggingFace
# !pip install --quiet sentencepiece #subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# Contructed random 50% dataset

In [3]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_from_disk
# from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np
from collections import defaultdict
import ipdb, re

pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

# Fraction for sample dataframe
proportion = 0.5

In [8]:
# # DOCTEAT
# df_train_tdm_docteat_f1_all_templates = pd.read_parquet('../data/df_train_tdm_docteat_f1_all_templates.parquet')
# df_dev_tdm_docteat_f1_all_templates = pd.read_parquet('../data/df_dev_tdm_docteat_f1_all_templates.parquet')

# df_train_tdm_docteat_f2_all_templates = pd.read_parquet('../data/df_train_tdm_docteat_f2_all_templates.parquet')
# df_dev_tdm_docteat_f2_all_templates = pd.read_parquet('../data/df_dev_tdm_docteat_f2_all_templates.parquet')

# df_train_tdms_docteat_f1_all_templates = pd.read_parquet('../data/df_train_tdms_docteat_f1_all_templates.parquet')
# df_dev_tdms_docteat_f1_all_templates = pd.read_parquet('../data/df_dev_tdms_docteat_f1_all_templates.parquet')

# df_train_tdms_docteat_f2_all_templates = pd.read_parquet('../data/df_train_tdms_docteat_f2_all_templates.parquet')
# df_dev_tdms_docteat_f2_all_templates = pd.read_parquet('../data/df_dev_tdms_docteat_f2_all_templates.parquet')

# # LONG
# df_train_tdm_long_f1_all_templates = pd.read_parquet('../data/df_train_tdm_long_f1_all_templates.parquet')
# df_dev_tdm_long_f1_all_templates = pd.read_parquet('../data/df_dev_tdm_long_f1_all_templates.parquet')

# df_train_tdm_long_f2_all_templates = pd.read_parquet('../data/df_train_tdm_long_f2_all_templates.parquet')
# df_dev_tdm_long_f2_all_templates = pd.read_parquet('../data/df_dev_tdm_long_f2_all_templates.parquet')

# df_train_tdms_long_f1_all_templates = pd.read_parquet('../data/df_train_tdms_long_f1_all_templates.parquet')
# df_dev_tdms_long_f1_all_templates = pd.read_parquet('../data/df_dev_tdms_long_f1_all_templates.parquet')

# df_train_tdms_long_f2_all_templates = pd.read_parquet('../data/df_train_tdms_long_f2_all_templates.parquet')
# df_dev_tdms_long_f2_all_templates = pd.read_parquet('../data/df_dev_tdms_long_f2_all_templates.parquet')


# df_train_f1_all_templates.to_parquet('../data/df_train_tdms_augmented_summarized_f1_all_templates.parquet')
# df_dev_f1_all_templates.to_parquet('../data/df_dev_tdms_augmented_summarized_f1_all_templates.parquet')
# df_train_f2_all_templates.to_parquet('../data/df_train_tdms_augmented_summarized_f2_all_templates.parquet')
# df_dev_f2_all_templates.to_parquet('../data/df_dev_tdms_augmented_summarized_f2_all_templates.parquet')


# AUGMENTED
df_train_tdms_augmented_f1_all_templates = pd.read_parquet('../data/df_train_tdms_augmented_summarized_f1_all_templates.parquet')
df_dev_tdm_augmented_f1_all_templates = pd.read_parquet('../data/df_dev_tdms_augmented_summarized_f1_all_templates.parquet')

df_train_tdms_augmented_f2_all_templates = pd.read_parquet('../data/df_train_tdms_augmented_summarized_f2_all_templates.parquet')
df_dev_tdms_augmented_f2_all_templates = pd.read_parquet('../data/df_dev_tdms_augmented_summarized_f2_all_templates.parquet')

df_train_tdms_augmented_f1_all_templates.head()

,prompt,answer
0,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
1,Read this and answer the question. If the ques...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
2,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
3,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
4,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."


In [5]:
# substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
# substring_to_match_squad_1

In [6]:
# Filter rows where the 'text' column contains the given substring
# substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
substring_to_match_squad_1 = 'Please answer a question about this article. If the question is unanswerable, say "unanswerable".'
substring_to_match_squad_2 = 'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".'
substring_to_match_squad_3 = '(If the question is unanswerable, say \"unanswerable\"'
substring_to_match_squad_4 = 'Try to answer this question if possible (otherwise reply \"unanswerable\")'
substring_to_match_squad_5 = 'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"):'
substring_to_match_squad_6 = 'Answer this question, if possible (if impossible, reply \"unanswerable\"):'
substring_to_match_squad_7 = 'What is the answer? (If it cannot be answered, return \"unanswerable\")'
substring_to_match_squad_8 = 'Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"):'

substring_to_match_drop_1 = 'Answer based on context:'
substring_to_match_drop_2 = 'Answer this question based on the article:'
# substring_to_match_drop_3 = ''
substring_to_match_drop_4 = 'Answer this question: '
substring_to_match_drop_5 = 'Read this article and answer this question'
substring_to_match_drop_6 = 'Based on the above article, answer a question.'
substring_to_match_drop_7 = 'Context: '

# DOCTEAT
## TDM
df_train_tdm_docteat_f1_all_templates['template'] = df_train_tdm_docteat_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdm_docteat_f1_all_templates['template'] = df_dev_tdm_docteat_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_train_tdm_docteat_f2_all_templates['template'] = df_train_tdm_docteat_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else 
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdm_docteat_f2_all_templates['template'] = df_dev_tdm_docteat_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

# TDMS
df_train_tdms_docteat_f1_all_templates['template'] = df_train_tdms_docteat_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdms_docteat_f1_all_templates['template'] = df_dev_tdms_docteat_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_train_tdms_docteat_f2_all_templates['template'] = df_train_tdms_docteat_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdms_docteat_f2_all_templates['template'] = df_dev_tdms_docteat_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

# LONG
## TDM
df_train_tdm_long_f1_all_templates['template'] = df_train_tdm_long_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdm_long_f1_all_templates['template'] = df_dev_tdm_long_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_train_tdm_long_f2_all_templates['template'] = df_train_tdm_long_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else 
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdm_long_f2_all_templates['template'] = df_dev_tdm_long_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

# TDMS
df_train_tdms_long_f1_all_templates['template'] = df_train_tdms_long_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdms_long_f1_all_templates['template'] = df_dev_tdms_long_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

df_train_tdms_long_f2_all_templates['template'] = df_train_tdms_long_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )
df_dev_tdms_long_f2_all_templates['template'] = df_dev_tdms_long_f2_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )


df_train_tdms_long_f1_all_templates.describe()

,prompt,answer,template
count,82680,82680,82680
unique,82335,3634,15
top,Title\tLaTeX Author Guidelines for CVPR Procee...,unanswerable,drop_7
freq,5,28080,5525


In [7]:
df_train_tdm_docteat_f1_all_templates_squad_1 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_2 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_3 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_4 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_5 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_6 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_7 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_squad_8 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdm_docteat_f1_all_templates_drop_1 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_drop_2 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_drop_3 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_drop_4 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_drop_5 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_drop_6 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdm_docteat_f1_all_templates_drop_7 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdm_docteat_f1_all_templates_50_percent = pd.concat([
    df_train_tdm_docteat_f1_all_templates_squad_1, 
    df_train_tdm_docteat_f1_all_templates_squad_2, 
    df_train_tdm_docteat_f1_all_templates_squad_3, 
    df_train_tdm_docteat_f1_all_templates_squad_4, 
    df_train_tdm_docteat_f1_all_templates_squad_5, 
    df_train_tdm_docteat_f1_all_templates_squad_6, 
    df_train_tdm_docteat_f1_all_templates_squad_7, 
    df_train_tdm_docteat_f1_all_templates_squad_8, 
    df_train_tdm_docteat_f1_all_templates_drop_1, 
    df_train_tdm_docteat_f1_all_templates_drop_2, 
    df_train_tdm_docteat_f1_all_templates_drop_3, 
    df_train_tdm_docteat_f1_all_templates_drop_4, 
    df_train_tdm_docteat_f1_all_templates_drop_5,
    df_train_tdm_docteat_f1_all_templates_drop_6,
    df_train_tdm_docteat_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdm_docteat_f1_all_templates_50_percent")
display(df_train_tdm_docteat_f1_all_templates_50_percent.describe())


df_dev_tdm_docteat_f1_all_templates_squad_1 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_2 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_3 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_4 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_5 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_6 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_7 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_squad_8 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdm_docteat_f1_all_templates_drop_1 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_drop_2 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_drop_3 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_drop_4 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_drop_5 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_drop_6 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdm_docteat_f1_all_templates_drop_7 = df_dev_tdm_docteat_f1_all_templates[df_dev_tdm_docteat_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdm_docteat_f1_all_templates_50_percent = pd.concat([
    df_dev_tdm_docteat_f1_all_templates_squad_1, 
    df_dev_tdm_docteat_f1_all_templates_squad_2, 
    df_dev_tdm_docteat_f1_all_templates_squad_3, 
    df_dev_tdm_docteat_f1_all_templates_squad_4, 
    df_dev_tdm_docteat_f1_all_templates_squad_5, 
    df_dev_tdm_docteat_f1_all_templates_squad_6, 
    df_dev_tdm_docteat_f1_all_templates_squad_7, 
    df_dev_tdm_docteat_f1_all_templates_squad_8, 
    df_dev_tdm_docteat_f1_all_templates_drop_1, 
    df_dev_tdm_docteat_f1_all_templates_drop_2, 
    df_dev_tdm_docteat_f1_all_templates_drop_3, 
    df_dev_tdm_docteat_f1_all_templates_drop_4, 
    df_dev_tdm_docteat_f1_all_templates_drop_5,
    df_dev_tdm_docteat_f1_all_templates_drop_6,
    df_dev_tdm_docteat_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdm_docteat_f1_all_templates_50_percent")
display(df_dev_tdm_docteat_f1_all_templates_50_percent.describe())


df_train_tdm_docteat_f2_all_templates_squad_1 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_2 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_3 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_4 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_5 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_6 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_7 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_squad_8 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdm_docteat_f2_all_templates_drop_1 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_drop_2 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_drop_3 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_drop_4 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_drop_5 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_drop_6 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdm_docteat_f2_all_templates_drop_7 = df_train_tdm_docteat_f2_all_templates[df_train_tdm_docteat_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdm_docteat_f2_all_templates_50_percent = pd.concat([
    df_train_tdm_docteat_f2_all_templates_squad_1, 
    df_train_tdm_docteat_f2_all_templates_squad_2, 
    df_train_tdm_docteat_f2_all_templates_squad_3, 
    df_train_tdm_docteat_f2_all_templates_squad_4, 
    df_train_tdm_docteat_f2_all_templates_squad_5, 
    df_train_tdm_docteat_f2_all_templates_squad_6, 
    df_train_tdm_docteat_f2_all_templates_squad_7, 
    df_train_tdm_docteat_f2_all_templates_squad_8, 
    df_train_tdm_docteat_f2_all_templates_drop_1, 
    df_train_tdm_docteat_f2_all_templates_drop_2, 
    df_train_tdm_docteat_f2_all_templates_drop_3, 
    df_train_tdm_docteat_f2_all_templates_drop_4, 
    df_train_tdm_docteat_f2_all_templates_drop_5,
    df_train_tdm_docteat_f2_all_templates_drop_6,
    df_train_tdm_docteat_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdm_docteat_f2_all_templates_50_percent")
display(df_train_tdm_docteat_f2_all_templates_50_percent.describe())


df_dev_tdm_docteat_f2_all_templates_squad_1 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_2 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_3 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_4 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_5 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_6 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_7 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_squad_8 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdm_docteat_f2_all_templates_drop_1 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_drop_2 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_drop_3 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_drop_4 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_drop_5 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_drop_6 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdm_docteat_f2_all_templates_drop_7 = df_dev_tdm_docteat_f2_all_templates[df_dev_tdm_docteat_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdm_docteat_f2_all_templates_50_percent = pd.concat([
    df_dev_tdm_docteat_f2_all_templates_squad_1, 
    df_dev_tdm_docteat_f2_all_templates_squad_2, 
    df_dev_tdm_docteat_f2_all_templates_squad_3, 
    df_dev_tdm_docteat_f2_all_templates_squad_4, 
    df_dev_tdm_docteat_f2_all_templates_squad_5, 
    df_dev_tdm_docteat_f2_all_templates_squad_6, 
    df_dev_tdm_docteat_f2_all_templates_squad_7, 
    df_dev_tdm_docteat_f2_all_templates_squad_8, 
    df_dev_tdm_docteat_f2_all_templates_drop_1, 
    df_dev_tdm_docteat_f2_all_templates_drop_2, 
    df_dev_tdm_docteat_f2_all_templates_drop_3, 
    df_dev_tdm_docteat_f2_all_templates_drop_4, 
    df_dev_tdm_docteat_f2_all_templates_drop_5,
    df_dev_tdm_docteat_f2_all_templates_drop_6,
    df_dev_tdm_docteat_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdm_docteat_f2_all_templates_50_percent")
display(df_dev_tdm_docteat_f2_all_templates_50_percent.describe())


df_train_tdms_docteat_f1_all_templates_squad_1 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_2 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_3 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_4 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_5 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_6 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_7 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_squad_8 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdms_docteat_f1_all_templates_drop_1 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_drop_2 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_drop_3 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_drop_4 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_drop_5 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_drop_6 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdms_docteat_f1_all_templates_drop_7 = df_train_tdms_docteat_f1_all_templates[df_train_tdms_docteat_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdms_docteat_f1_all_templates_50_percent = pd.concat([
    df_train_tdms_docteat_f1_all_templates_squad_1, 
    df_train_tdms_docteat_f1_all_templates_squad_2, 
    df_train_tdms_docteat_f1_all_templates_squad_3, 
    df_train_tdms_docteat_f1_all_templates_squad_4, 
    df_train_tdms_docteat_f1_all_templates_squad_5, 
    df_train_tdms_docteat_f1_all_templates_squad_6, 
    df_train_tdms_docteat_f1_all_templates_squad_7, 
    df_train_tdms_docteat_f1_all_templates_squad_8, 
    df_train_tdms_docteat_f1_all_templates_drop_1, 
    df_train_tdms_docteat_f1_all_templates_drop_2, 
    df_train_tdms_docteat_f1_all_templates_drop_3, 
    df_train_tdms_docteat_f1_all_templates_drop_4, 
    df_train_tdms_docteat_f1_all_templates_drop_5,
    df_train_tdms_docteat_f1_all_templates_drop_6,
    df_train_tdms_docteat_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdms_docteat_f1_all_templates_50_percent")
display(df_train_tdms_docteat_f1_all_templates_50_percent.describe())


df_dev_tdms_docteat_f1_all_templates_squad_1 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_2 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_3 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_4 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_5 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_6 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_7 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_squad_8 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdms_docteat_f1_all_templates_drop_1 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_drop_2 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_drop_3 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_drop_4 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_drop_5 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_drop_6 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdms_docteat_f1_all_templates_drop_7 = df_dev_tdms_docteat_f1_all_templates[df_dev_tdms_docteat_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdms_docteat_f1_all_templates_50_percent = pd.concat([
    df_dev_tdms_docteat_f1_all_templates_squad_1, 
    df_dev_tdms_docteat_f1_all_templates_squad_2, 
    df_dev_tdms_docteat_f1_all_templates_squad_3, 
    df_dev_tdms_docteat_f1_all_templates_squad_4, 
    df_dev_tdms_docteat_f1_all_templates_squad_5, 
    df_dev_tdms_docteat_f1_all_templates_squad_6, 
    df_dev_tdms_docteat_f1_all_templates_squad_7, 
    df_dev_tdms_docteat_f1_all_templates_squad_8, 
    df_dev_tdms_docteat_f1_all_templates_drop_1, 
    df_dev_tdms_docteat_f1_all_templates_drop_2, 
    df_dev_tdms_docteat_f1_all_templates_drop_3, 
    df_dev_tdms_docteat_f1_all_templates_drop_4, 
    df_dev_tdms_docteat_f1_all_templates_drop_5,
    df_dev_tdms_docteat_f1_all_templates_drop_6,
    df_dev_tdms_docteat_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdms_docteat_f1_all_templates_50_percent")
display(df_dev_tdms_docteat_f1_all_templates_50_percent.describe())

df_train_tdms_docteat_f2_all_templates_squad_1 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_2 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_3 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_4 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_5 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_6 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_7 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_squad_8 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdms_docteat_f2_all_templates_drop_1 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_drop_2 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_drop_3 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_drop_4 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_drop_5 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_drop_6 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdms_docteat_f2_all_templates_drop_7 = df_train_tdms_docteat_f2_all_templates[df_train_tdms_docteat_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdms_docteat_f2_all_templates_50_percent = pd.concat([
    df_train_tdms_docteat_f2_all_templates_squad_1, 
    df_train_tdms_docteat_f2_all_templates_squad_2, 
    df_train_tdms_docteat_f2_all_templates_squad_3, 
    df_train_tdms_docteat_f2_all_templates_squad_4, 
    df_train_tdms_docteat_f2_all_templates_squad_5, 
    df_train_tdms_docteat_f2_all_templates_squad_6, 
    df_train_tdms_docteat_f2_all_templates_squad_7, 
    df_train_tdms_docteat_f2_all_templates_squad_8, 
    df_train_tdms_docteat_f2_all_templates_drop_1, 
    df_train_tdms_docteat_f2_all_templates_drop_2, 
    df_train_tdms_docteat_f2_all_templates_drop_3, 
    df_train_tdms_docteat_f2_all_templates_drop_4, 
    df_train_tdms_docteat_f2_all_templates_drop_5,
    df_train_tdms_docteat_f2_all_templates_drop_6,
    df_train_tdms_docteat_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdms_docteat_f2_all_templates_50_percent")
display(df_train_tdms_docteat_f2_all_templates_50_percent.describe())

df_dev_tdms_docteat_f2_all_templates_squad_1 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_2 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_3 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_4 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_5 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_6 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_7 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_squad_8 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdms_docteat_f2_all_templates_drop_1 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_drop_2 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_drop_3 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_drop_4 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_drop_5 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_drop_6 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdms_docteat_f2_all_templates_drop_7 = df_dev_tdms_docteat_f2_all_templates[df_dev_tdms_docteat_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdms_docteat_f2_all_templates_50_percent = pd.concat([
    df_dev_tdms_docteat_f2_all_templates_squad_1, 
    df_dev_tdms_docteat_f2_all_templates_squad_2, 
    df_dev_tdms_docteat_f2_all_templates_squad_3, 
    df_dev_tdms_docteat_f2_all_templates_squad_4, 
    df_dev_tdms_docteat_f2_all_templates_squad_5, 
    df_dev_tdms_docteat_f2_all_templates_squad_6, 
    df_dev_tdms_docteat_f2_all_templates_squad_7, 
    df_dev_tdms_docteat_f2_all_templates_squad_8, 
    df_dev_tdms_docteat_f2_all_templates_drop_1, 
    df_dev_tdms_docteat_f2_all_templates_drop_2, 
    df_dev_tdms_docteat_f2_all_templates_drop_3, 
    df_dev_tdms_docteat_f2_all_templates_drop_4, 
    df_dev_tdms_docteat_f2_all_templates_drop_5,
    df_dev_tdms_docteat_f2_all_templates_drop_6,
    df_dev_tdms_docteat_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdms_docteat_f2_all_templates_50_percent")
display(df_dev_tdms_docteat_f2_all_templates_50_percent.describe())


df_train_tdm_docteat_f1_all_templates_50_percent


,prompt,answer,template
count,41340,41340,41340
unique,41332,2821,15
top,Read this: IEEE TRANSACTIONS ON PATTERN ANALYS...,unanswerable,squad_1
freq,2,13999,2756


df_dev_tdm_docteat_f1_all_templates_50_percent


,prompt,answer,template
count,17641,17641,17641
unique,17637,1338,15
top,PANDA: Adapting Pretrained Features for Anomal...,unanswerable,drop_7
freq,2,6031,1177


df_train_tdm_docteat_f2_all_templates_50_percent


...

,prompt,answer,template
count,17641,17641,17641
unique,17634,1549,15
top,Read this article and answer this question PAN...,unanswerable,drop_7
freq,2,6052,1177


df_train_tdms_docteat_f2_all_templates_50_percent


,prompt,answer,template
count,41341,41341,41341
unique,41335,3637,15
top,Read this article and answer this question PAN...,unanswerable,drop_7
freq,2,13993,2757


df_dev_tdms_docteat_f2_all_templates_50_percent


,prompt,answer,template
count,17640,17640,17640
unique,17640,1548,15
top,IEEE TRANSACTIONS ON KNOWLEDGE AND DATA ENGINE...,unanswerable,squad_1
freq,1,6035,1176


In [1]:
(41340+41341)/2

41340.5

In [2]:
(17641+17640)/2

17640.5

In [9]:
df_train_tdm_long_f1_all_templates_squad_1 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_2 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_3 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_4 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_5 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_6 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_7 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_squad_8 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdm_long_f1_all_templates_drop_1 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_drop_2 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_drop_3 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_drop_4 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_drop_5 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_drop_6 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdm_long_f1_all_templates_drop_7 = df_train_tdm_long_f1_all_templates[df_train_tdm_long_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdm_long_f1_all_templates_50_percent = pd.concat([
    df_train_tdm_long_f1_all_templates_squad_1, 
    df_train_tdm_long_f1_all_templates_squad_2, 
    df_train_tdm_long_f1_all_templates_squad_3, 
    df_train_tdm_long_f1_all_templates_squad_4, 
    df_train_tdm_long_f1_all_templates_squad_5, 
    df_train_tdm_long_f1_all_templates_squad_6, 
    df_train_tdm_long_f1_all_templates_squad_7, 
    df_train_tdm_long_f1_all_templates_squad_8, 
    df_train_tdm_long_f1_all_templates_drop_1, 
    df_train_tdm_long_f1_all_templates_drop_2, 
    df_train_tdm_long_f1_all_templates_drop_3, 
    df_train_tdm_long_f1_all_templates_drop_4, 
    df_train_tdm_long_f1_all_templates_drop_5,
    df_train_tdm_long_f1_all_templates_drop_6,
    df_train_tdm_long_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdm_long_f1_all_templates_50_percent")
display(df_train_tdm_long_f1_all_templates_50_percent.describe())


df_dev_tdm_long_f1_all_templates_squad_1 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_2 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_3 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_4 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_5 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_6 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_7 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_squad_8 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdm_long_f1_all_templates_drop_1 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_drop_2 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_drop_3 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_drop_4 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_drop_5 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_drop_6 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdm_long_f1_all_templates_drop_7 = df_dev_tdm_long_f1_all_templates[df_dev_tdm_long_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdm_long_f1_all_templates_50_percent = pd.concat([
    df_dev_tdm_long_f1_all_templates_squad_1, 
    df_dev_tdm_long_f1_all_templates_squad_2, 
    df_dev_tdm_long_f1_all_templates_squad_3, 
    df_dev_tdm_long_f1_all_templates_squad_4, 
    df_dev_tdm_long_f1_all_templates_squad_5, 
    df_dev_tdm_long_f1_all_templates_squad_6, 
    df_dev_tdm_long_f1_all_templates_squad_7, 
    df_dev_tdm_long_f1_all_templates_squad_8, 
    df_dev_tdm_long_f1_all_templates_drop_1, 
    df_dev_tdm_long_f1_all_templates_drop_2, 
    df_dev_tdm_long_f1_all_templates_drop_3, 
    df_dev_tdm_long_f1_all_templates_drop_4, 
    df_dev_tdm_long_f1_all_templates_drop_5,
    df_dev_tdm_long_f1_all_templates_drop_6,
    df_dev_tdm_long_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdm_long_f1_all_templates_50_percent")
display(df_dev_tdm_long_f1_all_templates_50_percent.describe())


df_train_tdm_long_f2_all_templates_squad_1 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_2 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_3 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_4 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_5 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_6 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_7 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_squad_8 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdm_long_f2_all_templates_drop_1 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_drop_2 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_drop_3 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_drop_4 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_drop_5 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_drop_6 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdm_long_f2_all_templates_drop_7 = df_train_tdm_long_f2_all_templates[df_train_tdm_long_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdm_long_f2_all_templates_50_percent = pd.concat([
    df_train_tdm_long_f2_all_templates_squad_1, 
    df_train_tdm_long_f2_all_templates_squad_2, 
    df_train_tdm_long_f2_all_templates_squad_3, 
    df_train_tdm_long_f2_all_templates_squad_4, 
    df_train_tdm_long_f2_all_templates_squad_5, 
    df_train_tdm_long_f2_all_templates_squad_6, 
    df_train_tdm_long_f2_all_templates_squad_7, 
    df_train_tdm_long_f2_all_templates_squad_8, 
    df_train_tdm_long_f2_all_templates_drop_1, 
    df_train_tdm_long_f2_all_templates_drop_2, 
    df_train_tdm_long_f2_all_templates_drop_3, 
    df_train_tdm_long_f2_all_templates_drop_4, 
    df_train_tdm_long_f2_all_templates_drop_5,
    df_train_tdm_long_f2_all_templates_drop_6,
    df_train_tdm_long_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdm_long_f2_all_templates_50_percent")
display(df_train_tdm_long_f2_all_templates_50_percent.describe())


df_dev_tdm_long_f2_all_templates_squad_1 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_2 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_3 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_4 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_5 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_6 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_7 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_squad_8 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdm_long_f2_all_templates_drop_1 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_drop_2 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_drop_3 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_drop_4 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_drop_5 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_drop_6 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdm_long_f2_all_templates_drop_7 = df_dev_tdm_long_f2_all_templates[df_dev_tdm_long_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdm_long_f2_all_templates_50_percent = pd.concat([
    df_dev_tdm_long_f2_all_templates_squad_1, 
    df_dev_tdm_long_f2_all_templates_squad_2, 
    df_dev_tdm_long_f2_all_templates_squad_3, 
    df_dev_tdm_long_f2_all_templates_squad_4, 
    df_dev_tdm_long_f2_all_templates_squad_5, 
    df_dev_tdm_long_f2_all_templates_squad_6, 
    df_dev_tdm_long_f2_all_templates_squad_7, 
    df_dev_tdm_long_f2_all_templates_squad_8, 
    df_dev_tdm_long_f2_all_templates_drop_1, 
    df_dev_tdm_long_f2_all_templates_drop_2, 
    df_dev_tdm_long_f2_all_templates_drop_3, 
    df_dev_tdm_long_f2_all_templates_drop_4, 
    df_dev_tdm_long_f2_all_templates_drop_5,
    df_dev_tdm_long_f2_all_templates_drop_6,
    df_dev_tdm_long_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdm_long_f2_all_templates_50_percent")
display(df_dev_tdm_long_f2_all_templates_50_percent.describe())


df_train_tdms_long_f1_all_templates_squad_1 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_2 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_3 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_4 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_5 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_6 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_7 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_squad_8 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdms_long_f1_all_templates_drop_1 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_drop_2 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_drop_3 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_drop_4 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_drop_5 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_drop_6 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdms_long_f1_all_templates_drop_7 = df_train_tdms_long_f1_all_templates[df_train_tdms_long_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdms_long_f1_all_templates_50_percent = pd.concat([
    df_train_tdms_long_f1_all_templates_squad_1, 
    df_train_tdms_long_f1_all_templates_squad_2, 
    df_train_tdms_long_f1_all_templates_squad_3, 
    df_train_tdms_long_f1_all_templates_squad_4, 
    df_train_tdms_long_f1_all_templates_squad_5, 
    df_train_tdms_long_f1_all_templates_squad_6, 
    df_train_tdms_long_f1_all_templates_squad_7, 
    df_train_tdms_long_f1_all_templates_squad_8, 
    df_train_tdms_long_f1_all_templates_drop_1, 
    df_train_tdms_long_f1_all_templates_drop_2, 
    df_train_tdms_long_f1_all_templates_drop_3, 
    df_train_tdms_long_f1_all_templates_drop_4, 
    df_train_tdms_long_f1_all_templates_drop_5,
    df_train_tdms_long_f1_all_templates_drop_6,
    df_train_tdms_long_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdms_long_f1_all_templates_50_percent")
display(df_train_tdms_long_f1_all_templates_50_percent.describe())


df_dev_tdms_long_f1_all_templates_squad_1 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_2 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_3 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_4 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_5 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_6 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_7 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_squad_8 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdms_long_f1_all_templates_drop_1 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_drop_2 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_drop_3 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_drop_4 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_drop_5 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_drop_6 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdms_long_f1_all_templates_drop_7 = df_dev_tdms_long_f1_all_templates[df_dev_tdms_long_f1_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdms_long_f1_all_templates_50_percent = pd.concat([
    df_dev_tdms_long_f1_all_templates_squad_1, 
    df_dev_tdms_long_f1_all_templates_squad_2, 
    df_dev_tdms_long_f1_all_templates_squad_3, 
    df_dev_tdms_long_f1_all_templates_squad_4, 
    df_dev_tdms_long_f1_all_templates_squad_5, 
    df_dev_tdms_long_f1_all_templates_squad_6, 
    df_dev_tdms_long_f1_all_templates_squad_7, 
    df_dev_tdms_long_f1_all_templates_squad_8, 
    df_dev_tdms_long_f1_all_templates_drop_1, 
    df_dev_tdms_long_f1_all_templates_drop_2, 
    df_dev_tdms_long_f1_all_templates_drop_3, 
    df_dev_tdms_long_f1_all_templates_drop_4, 
    df_dev_tdms_long_f1_all_templates_drop_5,
    df_dev_tdms_long_f1_all_templates_drop_6,
    df_dev_tdms_long_f1_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdms_long_f1_all_templates_50_percent")
display(df_dev_tdms_long_f1_all_templates_50_percent.describe())

df_train_tdms_long_f2_all_templates_squad_1 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_2 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_3 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_4 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_5 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_6 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_7 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_squad_8 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_train_tdms_long_f2_all_templates_drop_1 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_drop_2 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_drop_3 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_drop_4 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_drop_5 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_drop_6 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_train_tdms_long_f2_all_templates_drop_7 = df_train_tdms_long_f2_all_templates[df_train_tdms_long_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_train_tdms_long_f2_all_templates_50_percent = pd.concat([
    df_train_tdms_long_f2_all_templates_squad_1, 
    df_train_tdms_long_f2_all_templates_squad_2, 
    df_train_tdms_long_f2_all_templates_squad_3, 
    df_train_tdms_long_f2_all_templates_squad_4, 
    df_train_tdms_long_f2_all_templates_squad_5, 
    df_train_tdms_long_f2_all_templates_squad_6, 
    df_train_tdms_long_f2_all_templates_squad_7, 
    df_train_tdms_long_f2_all_templates_squad_8, 
    df_train_tdms_long_f2_all_templates_drop_1, 
    df_train_tdms_long_f2_all_templates_drop_2, 
    df_train_tdms_long_f2_all_templates_drop_3, 
    df_train_tdms_long_f2_all_templates_drop_4, 
    df_train_tdms_long_f2_all_templates_drop_5,
    df_train_tdms_long_f2_all_templates_drop_6,
    df_train_tdms_long_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_train_tdms_long_f2_all_templates_50_percent")
display(df_train_tdms_long_f2_all_templates_50_percent.describe())

df_dev_tdms_long_f2_all_templates_squad_1 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_1'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_2 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_2'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_3 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_3'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_4 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_4'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_5 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_5'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_6 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_6'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_7 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_7'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_squad_8 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='squad_8'].sample(frac=proportion)

df_dev_tdms_long_f2_all_templates_drop_1 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_1'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_drop_2 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_2'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_drop_3 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_3'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_drop_4 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_4'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_drop_5 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_5'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_drop_6 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_6'].sample(frac=proportion)
df_dev_tdms_long_f2_all_templates_drop_7 = df_dev_tdms_long_f2_all_templates[df_dev_tdms_long_f2_all_templates['template']=='drop_7'].sample(frac=proportion)

df_dev_tdms_long_f2_all_templates_50_percent = pd.concat([
    df_dev_tdms_long_f2_all_templates_squad_1, 
    df_dev_tdms_long_f2_all_templates_squad_2, 
    df_dev_tdms_long_f2_all_templates_squad_3, 
    df_dev_tdms_long_f2_all_templates_squad_4, 
    df_dev_tdms_long_f2_all_templates_squad_5, 
    df_dev_tdms_long_f2_all_templates_squad_6, 
    df_dev_tdms_long_f2_all_templates_squad_7, 
    df_dev_tdms_long_f2_all_templates_squad_8, 
    df_dev_tdms_long_f2_all_templates_drop_1, 
    df_dev_tdms_long_f2_all_templates_drop_2, 
    df_dev_tdms_long_f2_all_templates_drop_3, 
    df_dev_tdms_long_f2_all_templates_drop_4, 
    df_dev_tdms_long_f2_all_templates_drop_5,
    df_dev_tdms_long_f2_all_templates_drop_6,
    df_dev_tdms_long_f2_all_templates_drop_7
    ], ignore_index=True)
print("df_dev_tdms_long_f2_all_templates_50_percent")
display(df_dev_tdms_long_f2_all_templates_50_percent.describe())


df_train_tdm_long_f1_all_templates_50_percent


,prompt,answer,template
count,41340,41340,41340
unique,41226,2821,15
top,Title\tLaTeX Author Guidelines for CVPR Procee...,unanswerable,drop_7
freq,5,14046,2762


df_dev_tdm_long_f1_all_templates_50_percent


,prompt,answer,template
count,17640,17640,17640
unique,17596,1338,15
top,Title\tLaTeX Author Guidelines for ICCV Procee...,unanswerable,drop_7
freq,4,6126,1178


df_train_tdm_long_f2_all_templates_50_percent


...

,prompt,answer,template
count,17640,17640,17640
unique,17586,1549,15
top,Title\tLaTeX Author Guidelines for ICCV Procee...,unanswerable,drop_7
freq,4,6011,1178


df_train_tdms_long_f2_all_templates_50_percent


,prompt,answer,template
count,41342,41342,41342
unique,41178,3637,15
top,Read this and answer the question. If the ques...,unanswerable,drop_7
freq,6,14071,2762


df_dev_tdms_long_f2_all_templates_50_percent


,prompt,answer,template
count,17640,17640,17640
unique,17614,1548,15
top,Read this: Title\tLaTeX Author Guidelines for ...,unanswerable,drop_7
freq,2,6002,1180


In [10]:
df_train_tdm_docteat_f1_all_templates_50_percent.to_parquet('../data/df_train_tdm_docteat_f1_all_templates_50_percent.parquet')
df_dev_tdm_docteat_f1_all_templates_50_percent.to_parquet('../data/df_dev_tdm_docteat_f1_all_templates_50_percent.parquet')
df_train_tdm_docteat_f2_all_templates_50_percent.to_parquet('../data/df_train_tdm_docteat_f2_all_templates_50_percent.parquet')
df_dev_tdm_docteat_f2_all_templates_50_percent.to_parquet('../data/df_dev_tdm_docteat_f2_all_templates_50_percent.parquet')
df_train_tdms_docteat_f1_all_templates_50_percent.to_parquet('../data/df_train_tdms_docteat_f1_all_templates_50_percent.parquet')
df_dev_tdms_docteat_f1_all_templates_50_percent.to_parquet('../data/df_dev_tdms_docteat_f1_all_templates_50_percent.parquet')
df_train_tdms_docteat_f2_all_templates_50_percent.to_parquet('../data/df_train_tdms_docteat_f2_all_templates_50_percent.parquet')
df_dev_tdms_docteat_f2_all_templates_50_percent.to_parquet('../data/df_dev_tdms_docteat_f2_all_templates_50_percent.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdm_docteat_f1_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdm_docteat_f1_all_templates_50_percent.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdm_docteat_f2_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdm_docteat_f2_all_templates_50_percent.parquet')
    })
})
print("df_train_tdm_docteat_f1_all_templates_50_percent")
print(dataset)

dataset.save_to_disk("../data/LLLM_DOCTEAT_TDM_ALL_TEMPLATE_50_PERCENT")

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_docteat_f1_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_docteat_f1_all_templates_50_percent.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_docteat_f2_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_docteat_f2_all_templates_50_percent.parquet')
    })
})
print("df_train_tdms_docteat_f1_all_templates_50_percent")
print(dataset)

dataset.save_to_disk("../data/LLLM_DOCTEAT_TDMS_ALL_TEMPLATE_50_PERCENT")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

df_train_tdms_docteat_f1_all_templates_50_percent
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 41340
        })
        validation: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 17641
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 41341
        })
        validation: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 17640
        })
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/41340 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17641 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/41341 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17640 [00:00<?, ? examples/s]

In [11]:
df_train_tdm_long_f1_all_templates_50_percent.to_parquet('../data/df_train_tdm_long_f1_all_templates_50_percent.parquet')
df_dev_tdm_long_f1_all_templates_50_percent.to_parquet('../data/df_dev_tdm_long_f1_all_templates_50_percent.parquet')
df_train_tdm_long_f2_all_templates_50_percent.to_parquet('../data/df_train_tdm_long_f2_all_templates_50_percent.parquet')
df_dev_tdm_long_f2_all_templates_50_percent.to_parquet('../data/df_dev_tdm_long_f2_all_templates_50_percent.parquet')
df_train_tdms_long_f1_all_templates_50_percent.to_parquet('../data/df_train_tdms_long_f1_all_templates_50_percent.parquet')
df_dev_tdms_long_f1_all_templates_50_percent.to_parquet('../data/df_dev_tdms_long_f1_all_templates_50_percent.parquet')
df_train_tdms_long_f2_all_templates_50_percent.to_parquet('../data/df_train_tdms_long_f2_all_templates_50_percent.parquet')
df_dev_tdms_long_f2_all_templates_50_percent.to_parquet('../data/df_dev_tdms_long_f2_all_templates_50_percent.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdm_long_f1_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdm_long_f1_all_templates_50_percent.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdm_long_f2_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdm_long_f2_all_templates_50_percent.parquet')
    })
})
print("df_train_tdm_long_f1_all_templates_50_percent")
print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_TDM_ALL_TEMPLATE_50_PERCENT")

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_f1_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_f1_all_templates_50_percent.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_long_f2_all_templates_50_percent.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_long_f2_all_templates_50_percent.parquet')
    })
})
print("df_train_tdms_long_f1_all_templates_50_percent")
print(dataset)

dataset.save_to_disk("../data/LLLM_LONG_TDMS_ALL_TEMPLATE_50_PERCENT")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

df_train_tdms_long_f1_all_templates_50_percent
DatasetDict({
    fold1: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 41340
        })
        validation: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 17640
        })
    })
    fold2: DatasetDict({
        train: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 41342
        })
        validation: Dataset({
            features: ['prompt', 'answer', 'template'],
            num_rows: 17640
        })
    })
})


Saving the dataset (0/3 shards):   0%|          | 0/41340 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17640 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/41342 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17640 [00:00<?, ? examples/s]